# ACRO Tests

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
sys.path.insert(0, os.path.abspath(".."))

In [3]:
from acro import ACRO

### Instantiate ACRO

In [4]:
acro = ACRO()

DEBUG:acro:path: /home/unknown/work/current/ACRO/ACRO/acro/default.yaml
DEBUG:acro:config: {'output_template_file': 'ACRO output template v01b.xlsm', 'safe_SDC_set': 'Default', 'safe_tests': 'threshold nk pratio maxmin', 'safe_threshold': 10, 'safe_dof_threshold': 10, 'safe_nk_n': 2, 'safe_nk_k': 0.9, 'safe_pratio_p': 0.1, 'safe_SDC_variations': 'CIS ESS', 'safe_thresholdCIS': 60, 'safe_nk_nCIS': 5, 'safe_nk_kCIS': 0.5, 'safe_testsCIS': 'threshold nk pratio', 'safe_thresholdESS': 15, 'safe_dof_thresholdESS': 10, 'safe_nk_nESS': 2, 'safe_nk_kESS': 0.9, 'safe_pratio_pESS': 0.15, 'safe_testsESS': 'nk pratio'}


### Load test data

In [5]:
path = os.path.join("../data", "test_data.dta")
df = pd.read_stata(path)
df.head()

,charity,grant_type,index,year,inc_activity,inc_grants,inc_donations,inc_other,inc_total,total_costs,...,sh_staff_grants_given,sh_assets_grants_given,sh_income_balance,sh_staff_balance,sh_assets_balance,sh_income_assets,sh_staff_assets,sh_income_staff_costs,sh_assets_staff_costs,wgt
0,4Children,R,1.0,2011,2880902.0,9603182.0,91404.0,310947.0,12886435.0,12127472.0,...,NaN,NaN,0.072636,0.135971,0.767809,0.094602,0.177090,0.534203,5.646843,1.0
1,4Children,R,1.0,2014,6810520.0,18768904.0,58002.0,401879.0,26039304.0,25493796.0,...,NaN,NaN,0.057641,0.089150,1.001396,0.057560,0.089026,0.646561,11.232729,1.0
2,4Children,R,1.0,2015,7199403.0,21638036.0,132191.0,512654.0,29482284.0,32290108.0,...,NaN,NaN,-0.049619,-0.079828,-0.620210,0.080004,0.128711,0.621583,7.769365,1.0
3,4Children,R,1.0,2013,5573013.0,15194731.0,228844.0,267156.0,21263744.0,20989048.0,...,NaN,NaN,0.045740,0.068251,1.008259,0.045365,0.067692,0.670166,14.772749,1.0
4,4Children,R,1.0,2010,2056816.0,7335103.0,110256.0,424628.0,9926803.0,9769816.0,...,NaN,NaN,0.057696,0.122532,0.567539,0.101660,0.215901,0.470862,4.631749,1.0


### Pandas crosstab

In [6]:
table = pd.crosstab(df.year, df.grant_type)
table

grant_type,G,N,R,R/G
year,,,,
2010,15,59,71,8
2011,15,59,71,8
2012,15,59,71,8
2013,15,59,71,8
2014,15,59,71,8
2015,15,59,71,8


### ACRO crosstab

In [7]:
safe_table = acro.crosstab(df.year, df.grant_type)
safe_table

DEBUG:acro:crosstab()
DEBUG:acro:caller: safe_table = acro.crosstab(df.year, df.grant_type)
DEBUG:acro:get_aggfunc()
DEBUG:acro:aggfunc: None
DEBUG:acro:apply_suppression()
DEBUG:acro:outcome_df:
grant_type   G   N   R          R/G
year                               
2010        ok  ok  ok  threshold; 
2011        ok  ok  ok  threshold; 
2012        ok  ok  ok  threshold; 
2013        ok  ok  ok  threshold; 
2014        ok  ok  ok  threshold; 
2015        ok  ok  ok  threshold; 
DEBUG:acro:get_summary(): fail; threshold: 6 cells suppressed; 
DEBUG:acro:add_output(): output_0


grant_type,G,N,R,R/G
year,,,,
2010,15,59,71,NaN
2011,15,59,71,NaN
2012,15,59,71,NaN
2013,15,59,71,NaN
2014,15,59,71,NaN
2015,15,59,71,NaN


In [8]:
safe_table = acro.crosstab(df.year, df.grant_type, values=df.inc_grants, aggfunc="mean")
safe_table

DEBUG:acro:crosstab()
DEBUG:acro:caller: safe_table = acro.crosstab(df.year, df.grant_type, values=df.inc_grants, aggfunc="mean")
DEBUG:acro:get_aggfunc()
DEBUG:acro:aggfunc: <function mean at 0x7f9ed4160040>
DEBUG:acro:apply_suppression()
DEBUG:acro:outcome_df:
grant_type   G   N   R                            R/G
year                                                 
2010        ok  ok  ok  threshold; p-ratio; nk-rule; 
2011        ok  ok  ok                    threshold; 
2012        ok  ok  ok                    threshold; 
2013        ok  ok  ok                    threshold; 
2014        ok  ok  ok                    threshold; 
2015        ok  ok  ok                    threshold; 
DEBUG:acro:get_summary(): fail; threshold: 6 cells suppressed; p-ratio: 1 cells suppressed; nk-rule: 1 cells suppressed; 
DEBUG:acro:add_output(): output_1


grant_type,G,N,R,R/G
year,,,,
2010,9921906.0,0.000000,8402284.0,NaN
2011,8502247.0,124013.859375,7716880.0,NaN
2012,11458580.0,131859.062500,6958050.5,NaN
2013,13557147.0,147937.796875,7202273.5,NaN
2014,13748147.0,133198.250000,8277525.5,NaN
2015,11133433.0,146572.187500,10812888.0,NaN


### ACRO pivot_table

In [9]:
table = acro.pivot_table(
    df, index=["grant_type"], values=["inc_grants"], aggfunc=["mean", "std"]
)
table

DEBUG:acro:pivot_table()
DEBUG:acro:caller: table = acro.pivot_table(
DEBUG:acro:get_aggfuncs()
DEBUG:acro:get_aggfunc()
DEBUG:acro:aggfunc: <function mean at 0x7f9ed4160040>
DEBUG:acro:get_aggfunc()
DEBUG:acro:aggfunc: <function std at 0x7f9ed41601f0>
DEBUG:acro:aggfuncs: [<function mean at 0x7f9ed4160040>, <function std at 0x7f9ed41601f0>]
DEBUG:acro:apply_suppression()
DEBUG:acro:outcome_df:
                 mean        std
           inc_grants inc_grants
grant_type                      
G                  ok         ok
N                  ok         ok
R                  ok         ok
R/G                ok         ok
DEBUG:acro:get_summary(): pass
DEBUG:acro:add_output(): output_2


,mean,std
,inc_grants,inc_grants
grant_type,,
G,1.141279e+07,2.283220e+07
N,1.344319e+05,1.988737e+05
R,8.098502e+06,3.204495e+07
R/G,1.664827e+07,1.583532e+07


### ACRO OLS

In [10]:
new_df = df[["inc_activity", "inc_grants", "inc_donations", "total_costs"]]
new_df = new_df.dropna()

y = new_df.inc_activity
x = np.stack((new_df.inc_grants, new_df.inc_donations, new_df.total_costs), axis=1)
x = sm.add_constant(x)

# auto fit
results = acro.ols(y, x)
results.summary()

DEBUG:acro:ols() outcome: pass; dof=807.0 >= 10
DEBUG:acro:add_output(): output_3


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           inc_activity   R-squared:                       0.894
Model:                            OLS   Adj. R-squared:                  0.894
Method:                 Least Squares   F-statistic:                     2276.
Date:                Tue, 27 Sep 2022   Prob (F-statistic):               0.00
Time:                        15:43:44   Log-Likelihood:                -14493.
No. Observations:                 811   AIC:                         2.899e+04
Df Residuals:                     807   BIC:                         2.901e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.994e+05   5.31e+05      0.752      0.452   -6.43e+05    1.44e+06
x1            -0.8856      0.025    -36.128      0.000      -0.934      -0.837
x2            -0.6659      0.016    -40.905      0.000      -0.698      -0.634
x3             0.8318      0.011     78.937      0.000       0.811       0.853
==============================================================================
Omnibus:                     1348.637   Durbin-Watson:                   1.424
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1298161.546
Skew:                          10.026   Prob(JB):                         0.00
Kurtosis:                     197.973   Cond. No.                     1.06e+08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.06e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### List current ACRO ouputs

In [11]:
output = acro.get_outputs()
output

DEBUG:acro:get_outputs()


{'output_0': {'command': 'safe_table = acro.crosstab(df.year, df.grant_type)',
  'summary': 'fail; threshold: 6 cells suppressed; ',
  'outcome': '{"G":{"2010":"ok","2011":"ok","2012":"ok","2013":"ok","2014":"ok","2015":"ok"},"N":{"2010":"ok","2011":"ok","2012":"ok","2013":"ok","2014":"ok","2015":"ok"},"R":{"2010":"ok","2011":"ok","2012":"ok","2013":"ok","2014":"ok","2015":"ok"},"R\\/G":{"2010":"threshold; ","2011":"threshold; ","2012":"threshold; ","2013":"threshold; ","2014":"threshold; ","2015":"threshold; "}}',
  'output': {'G': {'2010': 15,
    '2011': 15,
    '2012': 15,
    '2013': 15,
    '2014': 15,
    '2015': 15},
   'N': {'2010': 59,
    '2011': 59,
    '2012': 59,
    '2013': 59,
    '2014': 59,
    '2015': 59},
   'R': {'2010': 71,
    '2011': 71,
    '2012': 71,
    '2013': 71,
    '2014': 71,
    '2015': 71},
   'R/G': {'2010': None,
    '2011': None,
    '2012': None,
    '2013': None,
    '2014': None,
    '2015': None}}},
 'output_1': {'command': 'safe_table = acro.c

### Remove an ACRO output before finalising

In [12]:
acro.remove_output("output_1")

DEBUG:acro:remove_output(): output_1 removed


### Finalise ACRO

In [13]:
output = acro.finalise()

DEBUG:acro:finalise()
DEBUG:acro:filename: results.json
DEBUG:acro:output: {
    "output_0": {
        "command": "safe_table = acro.crosstab(df.year, df.grant_type)",
        "summary": "fail; threshold: 6 cells suppressed; ",
        "outcome": "{\"G\":{\"2010\":\"ok\",\"2011\":\"ok\",\"2012\":\"ok\",\"2013\":\"ok\",\"2014\":\"ok\",\"2015\":\"ok\"},\"N\":{\"2010\":\"ok\",\"2011\":\"ok\",\"2012\":\"ok\",\"2013\":\"ok\",\"2014\":\"ok\",\"2015\":\"ok\"},\"R\":{\"2010\":\"ok\",\"2011\":\"ok\",\"2012\":\"ok\",\"2013\":\"ok\",\"2014\":\"ok\",\"2015\":\"ok\"},\"R\\/G\":{\"2010\":\"threshold; \",\"2011\":\"threshold; \",\"2012\":\"threshold; \",\"2013\":\"threshold; \",\"2014\":\"threshold; \",\"2015\":\"threshold; \"}}",
        "output": {
            "G": {
                "2010": 15,
                "2011": 15,
                "2012": 15,
                "2013": 15,
                "2014": 15,
                "2015": 15
            },
            "N": {
                "2010": 59,
      

In [14]:
output

{'output_0': {'command': 'safe_table = acro.crosstab(df.year, df.grant_type)',
  'summary': 'fail; threshold: 6 cells suppressed; ',
  'outcome': '{"G":{"2010":"ok","2011":"ok","2012":"ok","2013":"ok","2014":"ok","2015":"ok"},"N":{"2010":"ok","2011":"ok","2012":"ok","2013":"ok","2014":"ok","2015":"ok"},"R":{"2010":"ok","2011":"ok","2012":"ok","2013":"ok","2014":"ok","2015":"ok"},"R\\/G":{"2010":"threshold; ","2011":"threshold; ","2012":"threshold; ","2013":"threshold; ","2014":"threshold; ","2015":"threshold; "}}',
  'output': {'G': {'2010': 15,
    '2011': 15,
    '2012': 15,
    '2013': 15,
    '2014': 15,
    '2015': 15},
   'N': {'2010': 59,
    '2011': 59,
    '2012': 59,
    '2013': 59,
    '2014': 59,
    '2015': 59},
   'R': {'2010': 71,
    '2011': 71,
    '2012': 71,
    '2013': 71,
    '2014': 71,
    '2015': 71},
   'R/G': {'2010': None,
    '2011': None,
    '2012': None,
    '2013': None,
    '2014': None,
    '2015': None}}},
 'output_2': {'command': 'table = acro.pivot_